# LLM Serving (vLLM) + FastAPI + Ölçümler (Colab)
**Amaç:** LLM API’lerini, prompt yapılarını ve basit ölçümleri öğrenmek.  
**Bileşenler:** vLLM (OpenAI uyumlu server) + FastAPI (healthcheck/proxy) + basit yük testi & latency ölçümü.  
**Neden Colab?** Donanım/kurulum derdi olmadan hızlı denemeler.

> Bu notebook 7/24 servis için değil, **öğrenme ve portföy** içindir.


## 0) Ortam & GPU Kontrolü
**Neden?** vLLM GPU ile çok daha hızlıdır. Bu hücre, GPU hazır mı gösterir.

In [7]:
!nvidia-smi
# vLLM portu takılıysa kapat
!fuser -k 8000/tcp || true
# muhtemel eski servisleri sonlandır
#!pkill -f "vllm.entrypoints.openai.api_server" || true # this section fixes
#!pkill -f "uvicorn" || true
#!pip uninstall -y tensorflow tensorflow-estimator keras tensorboard tensorboard-data-server


Thu Aug 14 13:57:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 1) (Opsiyonel) Model Cache için Drive Mount
**Neden?** Oturum sonlanınca modeller silinir. Drive'a cache alırsanız tekrar indirmez.

In [1]:
USE_DRIVE_CACHE = True # if you want use old cache set false

if USE_DRIVE_CACHE:
  from google.colab import drive
  drive.mount('/content/drive')
  import os
  os.environ["HF_HOME"] = "/content/drive/MyDrive/hf_cache"
  !mkdir -p $HF_HOME
else:
  print("Cache disabled. HF_HOME is default.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2) Kurulum
**Neden?** vLLM (OpenAI uyumlu server), FastAPI (proxy/healthcheck), httpx (istemci), ölçüm için numpy/pandas.

In [2]:
!pip install "vllm>=0.5.0" fastapi uvicorn httpx numpy pandas rich nest_asyncio


In [3]:
!pip install torch==2.6.0
!pip install "vllm>=0.5.0" fastapi uvicorn httpx numpy pandas rich nest_asyncio

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvjitlink_cu

## 3) Model Seçimi
**Neden?** Öğrenme için küçük/orta model yeterli. Varsayılan: TinyLlama (1.1B).  
Alternatif (daha kaliteli): Phi-3-mini (3.8B) — T4 16GB'ta da genelde çalışır.

In [4]:
#MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Default
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct" # Alternative (better quality, heavier)
MAX_MODEL_LEN = 2048 # prompt + output upper bound
print(f"Using model {MODEL_ID} with max length {MAX_MODEL_LEN}")

Using model microsoft/Phi-3-mini-4k-instruct with max length 2048


In [5]:
import subprocess, sys, time, os

# önce olası eski süreçleri indir
os.system("fuser -k 8000/tcp || true")

MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
cmd = [
    sys.executable, "-m", "vllm.entrypoints.openai.api_server",
    "--model", MODEL_ID,
    "--max-model-len", "2048",
    "--host", "127.0.0.1",
    "--port", "8000",
    "--dtype", "float16",
    "--gpu-memory-utilization", "0.85",   # bir tık düşürdük
    "--swap-space", "8",                  # NVMe swap (Colab’ta disk) - OOM'a tampon
]
print("Running:", " ".join(cmd))
with open("vllm.log", "w") as f:
    proc = subprocess.Popen(cmd, stdout=f, stderr=subprocess.STDOUT)
time.sleep(8)
!tail -n 60 vllm.log


Running: /usr/bin/python3 -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-mini-4k-instruct --max-model-len 2048 --host 127.0.0.1 --port 8000 --dtype float16 --gpu-memory-utilization 0.85 --swap-space 8
INFO 08-14 13:59:07 [__init__.py:235] Automatically detected platform cuda.


## 4) vLLM OpenAI-uyumlu Server'ı Başlat
**Neden?** OpenAI API şemasını taklit eden bir endpoint açar: `/v1/chat/completions`.

In [6]:
import subprocess ,os ,time ,requests ,sys
# Initialize the VLLM server in the backend
cmd = [
    sys.executable, "-m", "vllm.entrypoints.openai.api_server",
    "--model", MODEL_ID,
    "--max-model-len", str(MAX_MODEL_LEN),
    "--host", "127.0.0.1",
    "--port", "8000",
    "--dtype", "float16",
    "--gpu-memory-utilization", "0.9",
]
print("Running:", " ".join(cmd))
vllm_proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT , text=True)
# Wait for the server to start
time.sleep(2)

for i in range(180):
  try:
    r = requests.get("http://127.0.0.1:8000/v1/models", timeout=1)
    if r.status_code == 200:
      print("✅ vLLM ready.")
      break
  except Exception as e:
    pass
  time.sleep(1)
else:
  print("⚠️ vLLM was ready late. Last 2000 characters log:")
  try:
    out = vllm_proc.stdout.read()
    print(out[-2000:])
  except Exception as e:
    print(e)


Running: /usr/bin/python3 -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-mini-4k-instruct --max-model-len 2048 --host 127.0.0.1 --port 8000 --dtype float16 --gpu-memory-utilization 0.9
✅ vLLM ready.


### Hızlı Duman Testi (Doğrudan vLLM'e)
**Neden?** Server yanıt veriyor mu ve modelden çıktı geliyor mu kontrol.

In [7]:
import requests , json
payload = {
    "model": MODEL_ID,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": " Introduce yourself in two sentences"},
    ],
    "max_tokens": 128,

}
r = requests.post("http://127.0.0.1:8000/v1/chat/completions", json=payload, timeout=30)
print("Status:",r.status_code)
print(r.json())

Status: 200
{'id': 'chatcmpl-3a8a1b5eba70488194a31f9f45e69149', 'object': 'chat.completion', 'created': 1755180008, 'model': 'microsoft/Phi-3-mini-4k-instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': " Hello, my name's Sam, and I'm a knowledgeable AI developed by Microsoft to assist with a wide range of tasks. I'm programmed to follow ethical guidelines and help users responsibly.", 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': 32007}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 18, 'total_tokens': 66, 'completion_tokens': 48, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'kv_transfer_params': None}


## 5) FastAPI Proxy + Healthcheck
**Neden?** Kendi servisimizi (healthz, logging, yetkilendirme, vb.) OpenAI şemasına bağlamak.

In [8]:
proxy_code = '''
from fastapi import FastAPI, Request, HTTPException
import httpx, os, time

VLLM_BASE = os.getenv("VLLM_BASE", "http://127.0.0.1:8000")
VLLM_CHAT = f"{VLLM_BASE}/v1/chat/completions"
VLLM_MODELS = f"{VLLM_BASE}/v1/models"

# İstersen el yazısı set edebilirsin; boşsa startup'ta /v1/models'ten alınır
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "")

app = FastAPI(title="LLM Proxy API")

@app.on_event("startup")
async def _startup():
    global DEFAULT_MODEL
    # Backend model listesini al
    async with httpx.AsyncClient(timeout=10) as c:
        r = await c.get(VLLM_MODELS)
        r.raise_for_status()
        data = r.json()
    models = [m.get("id") for m in data.get("data", []) if "id" in m]
    if not models:
        raise RuntimeError("vLLM arkasında model bulunamadı (/v1/models boş).")
    # DEFAULT_MODEL belirtilmediyse ilkini seç
    if not DEFAULT_MODEL:
        DEFAULT_MODEL = models[0]
    print(f"[proxy] served_models={models} DEFAULT_MODEL={DEFAULT_MODEL}")

@app.get("/healthz")
def healthz():
    return {"status": "ok", "default_model": DEFAULT_MODEL}

@app.get("/models")
async def models():
    async with httpx.AsyncClient(timeout=10) as c:
        r = await c.get(VLLM_MODELS)
    return r.json()

@app.post("/v1/chat/completions")
async def chat_completions(request: Request):
    if not DEFAULT_MODEL:
        raise HTTPException(503, "Backend model hazır değil.")
    payload = await request.json()

    # Model alanını normalize et
    m = payload.get("model")
    if m in (None, "", "local", "default"):
        payload["model"] = DEFAULT_MODEL

    t0 = time.perf_counter()
    async with httpx.AsyncClient(timeout=60) as c:
        r = await c.post(VLLM_CHAT, json=payload)
        data = r.json()
    dt = time.perf_counter() - t0

    usage = data.get("usage", {})
    print(f"[proxy] latency={dt:.3f}s prompt={usage.get('prompt_tokens')} "
          f"completion={usage.get('completion_tokens')} total={usage.get('total_tokens')}")
    return data
'''

with open("main.py", "w", encoding="utf-8") as f:
    f.write(proxy_code)

print("main.py yazıldı.")


main.py yazıldı.


In [10]:
import requests, time
for i in range(10):
    try:
        r = requests.get("http://127.0.0.1:8001/healthz", timeout=1)
        print("HTTP", r.status_code, r.text)
        break
    except Exception as e:
        print("retry...", e)
        time.sleep(0.5)

HTTP 200 {"status":"ok","default_model":"microsoft/Phi-3-mini-4k-instruct"}


In [9]:
import subprocess, time, requests, sys, os, signal

# Önce 8001 portunu kullanan süreç varsa öldür
try:
    os.system("fuser -k 8001/tcp || true")
except Exception as e:
    print("Port temizleme sırasında hata:", e)

# Uvicorn'u arka planda başlat
uvicorn_proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "main:app", "--host", "127.0.0.1", "--port", "8001", "--log-level", "info"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Hazır olmasını bekle
for i in range(30):
    try:
        r = requests.get("http://127.0.0.1:8001/healthz", timeout=1)
        if r.status_code == 200:
            print("✅ FastAPI proxy 8001 portunda hazır.")
            break
    except Exception:
        pass
    time.sleep(1)
else:
    print("⚠️ FastAPI başlatılamadı. Son 2000 karakter log:")
    try:
        out = uvicorn_proc.stdout.read()
        print(out[-2000:])
    except Exception as e:
        print("Log okunamadı:", e)


✅ FastAPI proxy 8001 portunda hazır.


### Proxy Üzerinden Test (OpenAI uyumlu)
**Neden?** Üretimde vLLM'i doğrudan değil, kendi servisimiz üzerinden çağırırız.

In [11]:
import requests, json
payload = {
    "model": "local", # we dont use proxy side but left it as per OpenAI scheme
    "messages": [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Introduce yourself in three items"},
    ],
    "max_tokens": 128,
}
r = requests.post("http://127.0.0.1:8001/v1/chat/completions", json=payload, timeout=30)
print("Status:",r.status_code)
print(r.json())
print(json.dumps(payload, indent=2)[:2000])

Status: 200
{'id': 'chatcmpl-10dd44f041cc4db4b564aab65133c8e6', 'object': 'chat.completion', 'created': 1755180021, 'model': 'microsoft/Phi-3-mini-4k-instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': " - Hi there! My name is Microsoft, created by Microsoft Corporation. I'm a large language model.", 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': 32007}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 17, 'total_tokens': 40, 'completion_tokens': 23, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'kv_transfer_params': None}
{
  "model": "local",
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful assistant"
    },
    {
      "role": "user",
      "content": "Introduce yourself in three items"
    }
  ],
  "max_tokens": 128
}


## 6) Basit Yük Testi & Latency Ölçümü
**Neden?** p50/p95/p99 gecikmeyi ve token/sn gibi değerleri görmek, sistemin sınırlarını anlamak.

In [12]:
import asyncio, httpx, time, statistics, numpy as np, pandas as pd, json

TARGET_URL = "http://127.0.0.1:8001/v1/chat/completions"
CONCURRENCY = 10 # number of concurrent requests
TOTAL_REQUESTS = 50
MAX_TOKENS = 128

payload = [
    {
        "model": "local",
        "messages": [
            {"role": "system", "content": "You are a concise assistant."},
            {"role": "user", "content": "Where is the capital of Turkey? Answer shortly"},
        ],
        "max_tokens": MAX_TOKENS,
    }
]

latencies = []
tokens_out= []
errors = 0

async def one_call(session, idx):
  global errors, latencies, tokens_out
  t0 = time.perf_counter()
  try:
    r = await session.post(TARGET_URL, json=payload[0]) # Use payload[0] to get the dictionary
    data = r.json() # Call the json() method
    dt = time.perf_counter() - t0
    latencies.append(dt)
    usage = data.get("usage", {})
    tokens_out.append(usage.get("completion_tokens", 0)) # Append 0 if usage or completion_tokens is missing
  except Exception as e:
    print(f"Error {idx}: {e}")
    errors += 1
    latencies.append(time.perf_counter() - t0) # Append latency even on error
    tokens_out.append(0) # Append 0 tokens on error

async def run_load():
  async with httpx.AsyncClient(timeout=60) as session:
    sem = asyncio.Semaphore(CONCURRENCY)
    async def bound_call(i):
      async with sem:
        await one_call(session, i)
    await asyncio.gather(*(bound_call(i) for i in range(TOTAL_REQUESTS)))

await run_load()

  #sum of metrics
if latencies:
  p50, p90, p95, p99 = np.percentile(latencies, [50, 90, 95, 99])
  avg = sum(latencies) / len(latencies)
  tok = sum(tokens_out)
  total_time = sum(latencies)
  tok_per_sec = (tok / total_time) if total_time > 0 else 0.0
  df = pd.DataFrame({"latency_s": latencies, "completion_tokens": tokens_out})
  print("Request count:", len(latencies),"Errors:", errors)
  print(f"p50={p50:.3f}s p90={p90:.3f}s p95={p95:.3f}s p99={p99:.3f}s avg={avg:.3f}s")
  print(f"Total completion tokens: {tok} | Average tok/request= {(tok/len(latencies)):.3f} | Max tok/request= {max(tokens_out)}")
  print(f"Approximately throughput: {tok_per_sec:.3f} tokens/s")
  display(df.head())
else:
  print("No data to process.")

Request count: 50 Errors: 0
p50=0.583s p90=0.805s p95=0.898s p99=0.985s avg=0.573s
Total completion tokens: 251 | Average tok/request= 5.020 | Max tok/request= 10
Approximately throughput: 8.755 tokens/s


,latency_s,completion_tokens
0,0.591431,4
1,0.594088,4
2,0.594778,5
3,0.600960,4
4,0.595217,4


### Sonuçları Kaydet (CSV)
**Neden?** Portföy/GitHub için ölçümleri dışa aktarmak.

In [13]:
import pandas as pd, numpy as np, time, json
#if df exist previous cell save it
try:
  df
  ts = int(time.time())
  path = f"/content/llm_metrics_{ts}.csv"
  df.to_csv(path, index=False)
  print(f"Saved to {path}")
except NameError:
  print("No data to save.")



Saved to /content/llm_metrics_1755180031.csv


## 8) Temizlik (İsteğe bağlı)
**Neden?** Arka plan process'lerini sonlandırmak.

In [14]:
try:
  import os, signal, time
  if 'uvicorn_proc' in globals() and uvicorn_proc.poll() is None:
    os.kill(uvicorn_proc.pid, signal.SIGTERM)
    time.sleep(1)
    print("Uvicorn process terminated.")
  if 'vllm_proc' in globals() and vllm_proc.poll() is None:
    os.kill(vllm_proc.pid, signal.SIGTERM)
    time.sleep(1)
    print("vLLM process terminated.")
except Exception as e:
  print("Temizlik sırasında hata:", e)

Uvicorn process terminated.
